In [11]:
from IPython.display import display, Markdown, clear_output
import json
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy as sp
from scipy.stats import multivariate_normal as mvn
import sympy as sym

fontsizes=18
plt.rcParams.update({'font.size': fontsizes})
plt.rcParams.update({"font.family": "serif"})
plt.rcParams.update({"mathtext.fontset" : "cm"})
plt.rcParams.update({'font.serif': 'Times New Roman'})
plt.close('all')

# Defining Widgets: 
equation = widgets.Text(
            value='2*x1-x2',
            placeholder='Type something',
            description='Equation:')
slider_mu2 = widgets.FloatSlider(
         value=2,
         min=0,
         max=5.0,
         step=0.1,
         description='$\mathrm{E}[x_2$]:',
         orientation='horizontal',
         readout=True,
         readout_format='2.2f',)

slider_sig2 = widgets.FloatSlider(
         value=slider_mu2.value*0.35,
         min=0,
         max=5.0,
         step=0.1,
         description='$\mathrm{D}[x_2$]:',
         orientation='horizontal',
         readout=True,
         readout_format='2.2f',)

button = widgets.Button(description='Refresh plot')

out = widgets.Output()

markdown_out = widgets.Output()

# Header text
info = Markdown("""# Plot limit state function and joint pdf of x1 and x2
        Change the mean value and standard deviation of x2 and click 'Refresh plot' to see the effects. 
        """)

mu_x1 = slider_mu2.max-1
sigma_x1 = (slider_mu2.max-1) * 0.15 

mu_x2 = slider_mu2.value
sigma_x2 = slider_sig2.value


rvlinspace = lambda mu,sigma: np.linspace(0,np.ceil(mu+5*sigma),100)

def on_button_clicked(b):
    with out:
        clear_output()
        x1,x2,u1,u2,beta_sym = sym.symbols('x1 x2 u1 u2 beta_sym')   # Declare variables as symbols
        g_sym = eval(equation.value)                                 # LSF (symbolic)
        gx1_sym = sym.solve(sym.Eq(g_sym,0),x2)[len(sym.solve(sym.Eq(g_sym,0),x2)) - 1] 
        gx1 = np.vectorize(sym.lambdify([x1], gx1_sym, modules=['math']))          # Convert to function 

        def plotlsf_x(ax1,mu_x2,sigma_x2,beta,alpha_1,alpha_2):
            dum_x1 = rvlinspace(mu_x1,sigma_x1)
            dum_x2 = rvlinspace(mu_x2,sigma_x2)
            X2, X1 = np.meshgrid(dum_x2,dum_x1, indexing='ij')
            MU = np.array([mu_x2,mu_x1])
            CX = np.array([[sigma_x2**2, 0],
                   [0, sigma_x1**2]])
            pos_x = np.empty(X2.shape + (2,))
            pos_x[:, :, 0] = X2 
            pos_x[:, :, 1] = X1
            fx = mvn(MU,CX).pdf(pos_x)
            # Plot
            ax1.contour(X2, X1, fx)
            ax1.plot(gx1(dum_x1),dum_x1)
            x1_des = alpha_1*beta*sigma_x1 + mu_x1
            x2_des = alpha_2*beta*sigma_x2 + mu_x2
            ax1.plot(np.array([mu_x2,x2_des]),np.array([mu_x1,x1_des]),':or')
            ax1.set_title('X-space')
            ax1.set_xlabel('$X_2$')
            ax1.set_ylabel('$X_1$')
            ax1.set_aspect('equal')
            ax1.set_xlim(0,slider_sig2.max+3)
            ax1.set_xticks(np.arange(0,slider_sig2.max+3+1,1))
            ax1.set_yticks(np.arange(0,dum_x1[-1]+1,1))
            ax1.set_ylim(0,dum_x1[-1])
            plt.tight_layout()
            plt.show()
            return

        def plotlsf_u(ax1,mu_x2,sigma_x2):
            # U-space
            gu_sym = g_sym.subs(x1,u1*sigma_x1+mu_x1).subs(x2,u2*sigma_x2+mu_x2)
            gu1_sym = sym.solve(sym.Eq(gu_sym,0),u2)[len(sym.solve(sym.Eq(gu_sym,0),u2))-1]
            gu1 = np.vectorize(sym.lambdify([u1], gu1_sym, modules=['math']))
            dg_du1 = sym.diff(gu_sym,u1)
            dg_du2 = sym.diff(gu_sym,u2)
            k = (dg_du1**2 + dg_du1**2)**0.5
            alpha_1V = [-1/np.sqrt(2)]
            alpha_2V = [1/np.sqrt(2)]
            a1next = -dg_du1/k
            a2next = -dg_du2/k
            tol = 0.00001
            diff_i = tol+1
            sol = []
            while diff_i > tol:
                # LSF u-space at iteration i
                gab = gu_sym.subs([(u1,alpha_1V[-1]*beta_sym),(u2,alpha_2V[-1]*beta_sym)])
                sol.append(float(sym.solve(gab, beta_sym)[0]))
                #  Design point x-space at iteration i
                # Estimate new alpha vector
                alpha_1V.append(a1next.subs(u2,alpha_2V[-1]*sol[-1]).subs(u1,alpha_1V[-1]*sol[-1]))
                alpha_2V.append(a2next.subs(u2,alpha_2V[-1]*sol[-1]).subs(u1,alpha_1V[-1]*sol[-1]))
                if len(sol)>2:
                    diff_i = abs(sol[-1] - sol[-2]) 
            beta = float(sol[-1])
            alpha_1 = alpha_1V[-1]
            alpha_2 = alpha_2V[-1]
            u = np.linspace(-5,5,100)
            U2, U1 = np.meshgrid(u,u, indexing='ij')
            pos_u = np.empty(U2.shape + (2,))
            pos_u[:, :, 0] = U2
            pos_u[:, :, 1] = U1
            fu = mvn(np.array([0,0]),np.array([[1, 0],[0, 1]])).pdf(pos_u)
            # Plot
            ax1.contour(U2, U1, fu)
            ax1.plot(gu1(u),u)
            ax1.plot(np.array([0,alpha_2*beta]),np.array([0,alpha_1*beta]),':or')
            ax1.set_aspect('equal')
            ax1.set_xlim(-5,5)
            ax1.set_ylim(-5,5)
            ax1.set_title('U-space')
            ax1.set_xlabel('$U_2$')
            ax1.set_ylabel('$U_1$')
            ax1.set_xticks(np.arange(-5,6,5))
            ax1.set_yticks(np.arange(-5,6,5))
            plt.text(5.1,1,'$P_f = {pf:.2e}$\n$\\beta = {b:2.2f}$'.format(pf=sp.stats.norm.cdf(-beta),b=beta))
            plt.tight_layout()
            plt.show()   
            return beta,alpha_1,alpha_2

        fig, ax2 = plt.subplots()
        beta,alpha_1,alpha_2 = plotlsf_u(ax2,slider_mu2.value,slider_sig2.value)
        fig, ax1 = plt.subplots()
        plotlsf_x(ax1,slider_mu2.value,slider_sig2.value,beta,alpha_1,alpha_2)
        
button.on_click(on_button_clicked)


grapher = widgets.VBox([equation,slider_mu2,slider_sig2, button,out])
display(info, grapher)

# Plot limit state function and joint pdf of x1 and x2
        Change the mean value and standard deviation of x2 and click 'Refresh plot' to see the effects. 
        